## Homework

The goal of this homework is to get familiar with MLflow.


## Q1. Install MLflow

To get started with MLflow you'll need to install the MLflow Python package.

For this we recommend creating a separate Python environment, for example, you can use [conda environments](https://docs.conda.io/projects/conda/en/latest/user-guide/getting-started.html#managing-envs), and then install the package there with `pip` or `conda`.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

In [2]:
!mlflow --version

mlflow, version 1.26.1


In [3]:
!mlflow

Usage: mlflow [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  artifacts    Upload, list, and download artifacts from an MLflow...
  azureml      Serve models on Azure ML.
  db           Commands for managing an MLflow tracking database.
  deployments  Deploy MLflow models to custom targets.
  experiments  Manage experiments.
  gc           Permanently delete runs in the `deleted` lifecycle stage.
  models       Deploy MLflow models locally.
  run          Run an MLflow project from the given URI.
  runs         Manage runs.
  sagemaker    Serve models on SageMaker.
  server       Run the MLflow tracking server.
  ui           Launch the MLflow tracking UI for local viewing of run...


## Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset. 

Download the data for January, February and March 2021 in parquet format from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

Use the script `preprocess_data.py` located in the folder [`homework`](https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/02-experiment-tracking/homework) to preprocess the data.

The script will

- load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
- fit a `DictVectorizer` on the training set (January 2021 data),
- save the preprocessed datasets and the model to disk

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

How many files were saved to `OUTPUT_FOLDER`?

* 1
* 3
* 4
* 7

In [5]:
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet -O data/green_tripdata_2021-01.parquet
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet -O data/green_tripdata_2021-02.parquet
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-03.parquet -O data/green_tripdata_2021-03.parquet


--2022-05-29 20:43:50--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-01.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.110.230
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.110.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1333519 (1.3M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2021-01.parquet’

data/green_tripdata 100%[===================>]   1.27M  7.18MB/s    in 0.2s    

2022-05-29 20:43:50 (7.18 MB/s) - ‘data/green_tripdata_2021-01.parquet’ saved [1333519/1333519]

--2022-05-29 20:43:51--  https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2021-02.parquet
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.110.230
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.110.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1145679 (1.1M) [binary/octet-stream]
Saving to: ‘data/green_tripdata_2021-02.parquet’

data/green_tripdata 100%[===================

In [7]:
!python preprocess_data.py --raw_data_path ./data --dest_path ./output

In [15]:
!ls -lrth output

total 7.6M
-rw-rw-r-- 1 michael michael 299K May 29 20:51 dv.pkl
-rw-rw-r-- 1 michael michael 2.1M May 29 20:51 valid.pkl
-rw-rw-r-- 1 michael michael 2.6M May 29 20:51 train.pkl
-rw-rw-r-- 1 michael michael 2.7M May 29 20:51 test.pkl


## Q3. Train a model with autolog

We will train a random forest regressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. The script will load the datasets produced by the previous step, train the model on the training set and finally calculate the RMSE on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

How many parameters are automatically logged by MLflow?

* 19
* 17
* 10
* 20

In [18]:
!python train-ml_flow.py

2022/05/29 21:09:06 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2022/05/29 21:09:33 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/michael/miniconda3/envs/ml_ops/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."


## Q4. Launch the tracking server locally

We want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry. 

Your task is to

* launch the tracking server on your local machine
* select a SQLite db for the backend store and a folder called `artifacts` for the artifacts store

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to `backend-store-uri`, what else do you need to pass to properly configure the server?

* `default-artifact-root`
* `serve-artifacts`
* `artifacts-only`
* `artifacts-destination`

### Launch command: 

`mlflow ui --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./artifacts`

## Q5. Tune the hyperparameters of the model

Now let's try to reduce the validation error by tuning the hyperparameters of the random forest regressor using `hyperopt`.

We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to MLflow for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open the MLflow UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Important: don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization.
* the RMSE obtained on the validation set (February 2021 data).

What's the best validation RMSE that you got?

* 6.128
* 6.628
* 7.128
* 7.628

In [23]:
!python hpo-mlflow.py

2022/05/29 21:35:10 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
100%|█████████| 50/50 [09:10<00:00, 11.01s/trial, best loss: 6.6284257482044735]


In [28]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

client = MlflowClient()

HPO_EXPERIMENT_NAME = "random-forest-hyperopt"
log_top = 5

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment(HPO_EXPERIMENT_NAME)

experiment = client.get_experiment_by_name(HPO_EXPERIMENT_NAME)
runs = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=log_top,
    order_by=["metrics.rmse ASC"]
)

In [67]:
best_run = client.search_runs(experiment.experiment_id, order_by=["metrics.rmse ASC"], max_results=1)[0]
run_id = best_run.info.run_id
run_id

'6acf9172e69a4174a47354ba2a3febcd'

In [64]:
best_run.data.metrics['rmse']

6.6284257482044735

## Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good so we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2021 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE.
Tip 2: to register the model you can use the method `mlflow.register_model` and you will need to pass the right model_uri in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

* 6.1
* 6.55
* 7.93
* 15.1

In [69]:
model_uri = f"runs:/{run_id}/models"
mlflow.register_model(model_uri=model_uri, name="trip_duration_model")

Successfully registered model 'test'.
2022/05/29 22:20:41 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: test, version 1
Created version '1' of model 'test'.


<ModelVersion: creation_timestamp=1653862841062, current_stage='None', description='', last_updated_timestamp=1653862841062, name='test', run_id='6acf9172e69a4174a47354ba2a3febcd', run_link='', source='./artifacts/2/6acf9172e69a4174a47354ba2a3febcd/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [70]:
!python register_model-edit.py

2022/05/29 22:23:47 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2022/05/29 22:24:10 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/michael/miniconda3/envs/ml_ops/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'trip_duration_model'.
2022/05/29 22:25:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: trip_duration_model, version 1
Created version '1' of model 'trip_duration_model'.


## Submit the results

Submit your results here: https://forms.gle/9wXF5ntBA3FNe65L9

It's possible that your answers won't match exactly. If it's the case, select the closest one.


## Deadline

The deadline for submitting is 31 May 2022 (Tuesday) at 17:00 CET. After that, the form will be closed.


## Solution

The solution will be put here after the deadline.